In [ ]:
# Imports

from matplotlib import pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import cv2 as cv

In [ ]:
# Przygotowanie zdjęcia 

# Pobieramy aktualny katalog, w którym znajduje się skrypt
script_directory = os.getcwd()

# Konkatenacja ścieżki do pliku
img_path = os.path.join(script_directory, "ds", "24_0_2_20161219192221394.jpg.chip.jpg")

img = cv.imread(img_path)

# Zmiana rozdzielczości na taką, jaką ma nasz dataset 
img = cv.resize(img, (200, 200))

img = cv.cvtColor(img, cv.COLOR_RGB2BGR) # Żeby zdjęcie nie robiło się niebieskie

In [ ]:
# Wyświetl, żeby zobaczyć, czy poprawnie się wczytało
plt.imshow(img)
plt.show()

In [ ]:
# Konfiguracja klasy ImageDataGenerator modyfikującej dane
datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.05,
      horizontal_flip=True,
      vertical_flip=False,
      brightness_range=[0.5, 1.5],
      fill_mode='nearest')

* Wartość rotation_range określa stopnie (0–180) — zakres kątów, o które zostanie wykonany losowy obrót obrazów.
* Zakresy width_shift i height_shift określają ułamki całkowitej szerokości i wysokości obrazów, zakresy te określają ramy, w obrębie których przeprowadza się losowe pionowe i poziome przekształcenia obrazów.
* Parametr shear_range określa zakres losowego przycinania obrazu.
* Parametr zoom_range określa zakres losowego przybliżania fragmentów obrazów.
* Operacja horizontal_flip polega na losowym odbiciu połowy obrazu w płaszczyźnie poziomej — z przekształcenia tego warto korzystać wtedy, gdy nie ma założeń o horyzontalnej asymetrii obrazu (np. w przypadku prawdziwych zdjęć).
* Tryb fill_mode jest strategią wypełniania nowo utworzonych pikseli, które mogą powstać w wyniku obrotu lub przesunięcia.

Dokumentacja:
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

TODO: fill_mode='reflect' - do rozważenia 

In [ ]:
x = tf.keras.utils.img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
fig, axes = plt.subplots(2, 2, figsize=(8, 8))  # Tworzymy siatkę 2x2
for batch in datagen.flow(x, batch_size=1):
    ax = axes[i // 2, i % 2]  # Wybieramy odpowiednią komórkę na siatce
    imgplot = ax.imshow(tf.keras.utils.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

# Wyświetl wyniki augmentacji przykładowego zdjęcia
plt.show()


Puszczenie powyższego kodu wielokrotnie zwróci różne wyniki, ponieważ klasa tf.keras.preprocessing.image.ImageDataGenerator zawiera w sobie losowość

Powyższa część pokazywała, jak wygląda augmentacja na przykładzie jednego obrazka. W dalszej części wyjaśnienie dotyczące zastosowania przy trenowaniu danych.

## Augmentacja zbioru treningowego
Ważne: __Nie powinno się modyfikować danych walidacyjnych__

In [ ]:
# Przykład wywołania:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

# Sposób użycia
Metoda fit_generator — jest to odpowiednik metody fit stosowany w przypadku generatorów danych. Metoda ta oczekuje zdefiniowania w pierwszym argumencie generatora, który w nieskończoność będzie zwracał wsady danych wejściowych i ich etykiet. Dane są generowane w nieskończoność, a więc model Keras musi wiedzieć, ile próbek ma pobrać z generatora przed zakończeniem epoki. Służy do tego argument steps_per_epoch: po pobraniu liczby wsadów określanej przez wartość tego argumentu (tj. po wykonaniu odpowiedniej liczby kroków spadku gradientu) proces dopasowywania modelu przejdzie do kolejnej epoki. W naszym przypadku wsady składają się z 20 próbek, a więc musimy wygenerować 100 wsadów w celu wytrenowania modelu na 2000 próbek.

Korzystając z metody fit_generator, możemy — podobnie jak w przypadku metody fit — przekazać argument validation_data. Argument ten może być generatorem danych, a także krotką tablic Numpy. W przypadku przekazania generatora w argumencie validation_data oczekuje się, że generator ten będzie zwracał wsady danych walidacyjnych w nieskończoność. W związku z tym należy zdefiniować wartość argumentu validation_steps określającą liczbę wsadów, która ma zostać pobrana z generatora danych walidacyjnych w celu przeprowadzenia walidacji.